In [2]:
import boto3
import time
import csv
from pathlib import Path

textractor = boto3.client("textract", region_name="ap-southeast-1")

file_path = Path(r"D:\Engineer\Aws - sagemaker\sagemaker-mob\cth2.pdf")

if not file_path.exists():
    raise FileNotFoundError(f"The file {file_path} does not exist.")

with open(file_path, "rb") as document:
    imageBytes = bytearray(document.read())

response = textractor.analyze_document(
    Document={"Bytes": imageBytes}, FeatureTypes=["TABLES"]
)

print(response)

{'DocumentMetadata': {'Pages': 1}, 'Blocks': [{'BlockType': 'PAGE', 'Geometry': {'BoundingBox': {'Width': 1.0, 'Height': 1.0, 'Left': 0.0, 'Top': 0.0}, 'Polygon': [{'X': 3.2898975632633665e-07, 'Y': 0.0}, {'X': 1.0, 'Y': 2.8221265324646083e-07}, {'X': 1.0, 'Y': 1.0}, {'X': 0.0, 'Y': 1.0}]}, 'Id': 'e8893a25-7d5c-4c2d-8b8b-eac7b883c610', 'Relationships': [{'Type': 'CHILD', 'Ids': ['c8184bfb-d32c-492d-ba1e-ca2aa75284eb', 'bec61b69-b265-4406-a9fb-253c03dd2daf', '2c0cd115-13d0-4889-9215-30c321b3d0c7', '16f5643f-fcd4-4dfa-bc55-4cb9293285e2', '3f39bb89-9c27-4ecf-a806-fa5c0e1d6a39', '4b1acfd9-c2a4-42e6-a353-2aad215468e9', 'cdbb672a-28b5-469f-b205-343c4f91b629', '75650245-4a11-4fe1-849c-2dcdb588b679', '34f2608d-a14c-40e2-9c6f-396c478d1b7d', 'd9790345-52ab-4ae6-8326-381622c0bffd', 'f1418f06-d49c-496f-b7c2-f7daf8753489', '44f1a6b0-def9-4f5c-8f90-8b582e4b59c5', 'a2d25466-7f56-4402-9439-a10cf50b5c71', '2479b16f-2026-4ad9-a6dc-7b56cd6ae746', 'e0de66ad-0da1-48ef-af65-cb9f67842d2a', 'e595ec56-45f1-4bd

In [3]:
with open(file_path, "rb") as document:
    imageBytes = bytearray(document.read())

response = textractor.analyze_document(
    Document={"Bytes": imageBytes}, FeatureTypes=["TABLES"]
)

blocks = response["Blocks"]
tables = []

for block in blocks:
    if block["BlockType"] == "TABLE":
        table = []
        cell_map = {}
        for relationship in block["Relationships"]:
            if relationship["Type"] == "CHILD":
                for id in relationship["Ids"]:
                    cell_map[id] = None

        for cell in blocks:
            if cell["BlockType"] == "CELL" and cell["Id"] in cell_map:
                row_index = cell["RowIndex"]
                col_index = cell["ColumnIndex"]
                text = ""
                for relationship in cell["Relationships"]:
                    if relationship["Type"] == "CHILD":
                        for id in relationship["Ids"]:
                            for item in blocks:
                                if item["Id"] == id and item["BlockType"] == "WORD":
                                    text += item["Text"] + " "
                text = text.strip()
                cell_map[cell["Id"]] = (row_index, col_index, text)
                table.append((row_index, col_index, text))

        tables.append(table)

csv_file_path = Path(r"D:\Engineer\Aws - sagemaker\sagemaker-mob\extracted_table.csv")

with open(csv_file_path, mode="w", newline="", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)
    for table in tables:
        max_row = max(cell[0] for cell in table)
        max_col = max(cell[1] for cell in table)
        table_data = [["" for _ in range(max_col)] for _ in range(max_row)]

        for cell in table:
            row_index, col_index, text = cell
            table_data[row_index - 1][col_index - 1] = text

        writer.writerows(table_data)
        writer.writerow([]) 

print(f"Output {csv_file_path}")

Extracted table data has been saved to D:\Engineer\Aws - sagemaker\sagemaker-mob\extracted_table.csv
